> ライブラリ&データ読み込み

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

spark = SparkSession.builder \
        .appName("Data_wrangling") \
        .getOrCreate()

df_customer = spark.read.format("csv") \
    .option("inferSchema", "True") \
    .option("header", "True") \
    .option("sep", ",") \
    .load("./data/customer.csv")
df_category = spark.read.format("csv") \
    .option("inferSchema", "True") \
    .option("header", "True") \
    .option("sep", ",") \
    .load("./data/category.csv")
df_product = spark.read.format("csv") \
    .option("inferSchema", "True") \
    .option("header", "True") \
    .option("sep", ",") \
    .load("./data/product.csv")
df_receipt = spark.read.format("csv") \
    .option("inferSchema", "True") \
    .option("header", "True") \
    .option("sep", ",") \
    .load("./data/receipt.csv")
df_store = spark.read.format("csv") \
    .option("inferSchema", "True") \
    .option("header", "True") \
    .option("sep", ",") \
    .load("./data/store.csv")
df_geocode = spark.read.format("csv") \
    .option("inferSchema", "True") \
    .option("header", "True") \
    .option("sep", ",") \
    .load("./data/geocode.csv")

 > P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [ ]:
df_receipt.show(10)

 > P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [ ]:
df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount").show(10)

 > P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更して抽出すること。

In [ ]:
df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount").withColumnRenamed("sales_ymd", "sales_date").show(10)

 > P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
 > - 顧客ID（customer_id）が"CS018205000001"

In [ ]:
df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount") \
    .filter(F.col("customer_id") == "CS018205000001") \
    .show()

 > P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
 > - 顧客ID（customer_id）が"CS018205000001"
 > - 売上金額（amount）が1,000以上

In [ ]:
df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount") \
    .filter((F.col("customer_id") == "CS018205000001") & (F.col("amount")>=1000)) \
    .show()

 > P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
 > - 顧客ID（customer_id）が"CS018205000001"
 > - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
df_receipt.select("sales_ymd", "customer_id", "product_cd", "quantity", "amount") \
    .filter(F.col("customer_id") == "CS018205000001") \
    .filter((F.col("amount")>=1000) | (F.col("quantity") >= 5)) \
    .show()

 > P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
 > - 顧客ID（customer_id）が"CS018205000001"
 > - 売上金額（amount）が1,000以上2,000以下

In [ ]:
df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount") \
    .filter(F.col("customer_id") == "CS018205000001") \
    .filter((F.col("amount")>=1000) & (F.col("amount") <= 2000)) \
    .show()

 > P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
 > - 顧客ID（customer_id）が"CS018205000001"
 > - 商品コード（product_cd）が"P071401019"以外

In [ ]:
df_receipt.select("sales_ymd", "customer_id", "product_cd", "amount") \
    .filter((F.col("customer_id") == "CS018205000001") & (F.col("product_cd") != "P071401019")) \
    .show()

 > P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
 >
 > `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [ ]:
df_store.filter((F.col("prefecture_cd") != "13") & (F.col("floor_area") <= 900)) \
    .show()

 > P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [ ]:
df_store.filter(F.col("store_cd").rlike("^S14")) \
    .show(10)

 > P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [ ]:
df_customer.filter(F.col("customer_id").rlike("1$")) \
    .show(10)

 > P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [ ]:
df_store.filter(F.col("address").contains("横浜市")) \
    .show()

 > P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [ ]:
df_customer.filter(F.col("status_cd").rlike("^[A-F]")) \
    .show(10)

 > P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [ ]:
df_customer.filter(F.col("status_cd").rlike("[1-9]$")) \
    .show(10)

 > P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [ ]:
df_customer.filter((F.col("status_cd").rlike("^[A-F]")) & (F.col("status_cd").rlike("[1-9]$"))) \
    .show(10)

 > P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [ ]:
df_store.filter(F.col("tel_no").rlike("[0-9]{3}-[0-9]{3}-[0-9]{4}")) \
    .show()

 > P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [ ]:
df_customer.orderBy(F.asc("birth_day")) \
    .show(10)

 > P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [ ]:
df_customer.orderBy(F.desc("birth_day")) \
    .show(10)

 > P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [ ]:
window = Window.partitionBy("amount").orderBy(F.desc("amount"))
df_receipt.select("customer_id", "amount") \
    .withColumn("rank", F.rank().over(window)) \
    .show(10)

 > P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [ ]:
window = Window.partitionBy("amount").orderBy(F.desc("amount"))
df_receipt.select("customer_id", "amount") \
    .withColumn("rank", F.row_number().over(window)) \
    .show(10)

 > P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [ ]:
df_receipt.count()

 > P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [ ]:
df_receipt.select("customer_id") \
    .distinct() \
    .count()

 > P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [ ]:
df_receipt.groupBy("store_cd") \
    .agg({"amount":"sum", "quantity":"sum"}) \
    .show()

 > P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_receipt.groupBy("customer_id") \
    .agg({"sales_ymd": "max"}) \
    .show(10)

 > P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_receipt.groupBy("customer_id") \
    .agg({"sales_ymd": "min"}) \
    .show(10)

 > P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [ ]:
df_receipt.groupBy("customer_id") \
    .agg(F.min("sales_ymd"), F.max("sales_ymd")) \
    .filter(F.col("min(sales_ymd)") != F.col("max(sales_ymd)")) \
    .show(10)

 > P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupBy("store_cd") \
    .agg(F.avg("amount")) \
    .orderBy(F.desc("avg(amount)")) \
    .show(5)

 > P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupBy("store_cd") \
    .agg(F.median("amount")) \
    .orderBy(F.desc("median(amount)")) \
    .show(5)

 > P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [ ]:
df_receipt.groupBy("store_cd") \
    .agg(F.mode("product_cd")) \
    .show(10)

 > P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [ ]:
df_receipt.groupBy("store_cd") \
    .agg(F.variance("amount")) \
    .orderBy(F.desc("var_samp(amount)")) \
    .show(5)

 > P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

In [ ]:
df_receipt.groupBy("store_cd") \
    .agg(F.stddev("amount")) \
    .orderBy(F.desc("stddev(amount)")) \
    .show(5)

 > P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [ ]:
window = Window.partitionBy("amount").orderBy(F.desc("amount"))
df_receipt.select("amount") \
    .withColumn("percentile", F.percentile("amount", [0.0, 0.25, 0.5, 0.75, 1.0]).over(window)) \
    .show(truncate=False)

 > P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [ ]:
df_receipt.groupBy("store_cd") \
    .agg(F.avg("amount")) \
    .filter(F.col("avg(amount)") >= 330) \
    .show()

 > P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [ ]:
df_receipt.filter(F.col("customer_id").rlike("^(?!Z)")) \
    .groupBy("customer_id") \
    .agg(F.sum("amount")) \
    .agg(F.avg("sum(amount)"))\
    .show()

 > P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [ ]:
avg = df_receipt.filter(F.col("customer_id").rlike("^(?!Z)")).groupBy("customer_id").agg(F.sum("amount")).agg(F.avg("sum(amount)")).first()[0]
df_receipt.filter(F.col("amount") >= avg).show(10)

 > P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [ ]:
df_receipt.join(df_store, on="store_cd", how="inner") \
    .select(*df_receipt.columns+["store_name"]) \
    .show(10)

 > P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [ ]:
df_product.join(df_category, on=["category_major_cd", "category_medium_cd", "category_small_cd"], how="inner") \
    .select(*df_product.columns+["category_small_name"]) \
    .show(10)

 > P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [ ]:
df_customer.join(df_receipt, on="customer_id", how="inner") \
    .filter((F.col("gender_cd") == 1) & (F.col("customer_id").rlike("^(?!Z)"))) \
    .groupBy("customer_id") \
    .agg(F.sum("amount")) \
    .fillna(0) \
    .show(10)

 > P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [ ]:
df_sale_date = df_receipt.filter(F.col("customer_id").rlike("^(?!Z)")) \
                .groupBy("customer_id").agg(F.count("sales_ymd")) \
                .orderBy(F.desc("count(sales_ymd)")) \
                .limit(20)
df_sale_amount = df_receipt.filter(F.col("customer_id").rlike("^(?!Z)")) \
                .groupBy("customer_id").agg(F.sum("amount")) \
                .orderBy(F.desc("sum(amount)")) \
                .limit(20)
df_sale_date.join(df_sale_amount, how="outer", on="customer_id").show()

 > P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [ ]:
df_store.crossJoin(df_product).count()

 > P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [ ]:
window = Window.partitionBy().orderBy(F.asc("sales_ymd"))
df_receipt.groupBy("sales_ymd") \
    .agg(F.sum("amount")) \
    .withColumn("diff", F.col("sum(amount)") - F.lag("sum(amount)", 1).over(window)) \
    .show()